In [1]:
from transformers import ViTImageProcessor, ViTForImageClassification, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
import pandas as pd
import os
from tqdm import tqdm
import numpy as np 
from transformers import get_linear_schedule_with_warmup
import random
import numpy as np
import torch
import timm 

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Call this function at the start of your main function
set_seed()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



/opt/miniconda3/envs/MLEnv/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/miniconda3/envs/MLEnv/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2D1B8D5C-7891-3680-9CF9-F771AE880676> /opt/miniconda3/envs/MLEnv/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <CDAC6E34-8608-3E70-8B2F-32BCD38E90FB> /opt/miniconda3/envs/MLEnv/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
device

device(type='cpu')

In [3]:

groun_truths = pd.read_csv('/Users/jimmyhe/Desktop/KaggleCompetitions/ISISCANCER/MetaDataPlusProprocessed/ISIC_2019_Training_GroundTruth.csv')
groun_truths.columns




Index(['image', 'MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], dtype='object')

In [10]:
import geffnet
import torch
import torch.nn as nn
import OrderedDict

# Load the EfficientNet-B7 model without pre-trained weights
model = geffnet.create_model('efficientnet_b7', pretrained=False)
num_features = model.classifier.in_features  # Get the number of features in the classifier
model.classifier = nn.Linear(num_features, 2)  # Change the output to 2 classes for binary classification

# Load the state dictionary from the .pth file
state_dict = torch.load('./Melanoma_2020_models/9c_b7ns_1e_576_ext_15ep_oldfold_best_fold4.pth', map_location=torch.device('cpu'))

# Remove the 'module.' prefix from the state dictionary keys
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

# Now load the modified state dictionary into the model
model.load_state_dict(new_state_dict, strict=False)  # strict=False to ignore the classifier mismatch

_IncompatibleKeys(missing_keys=['conv_stem.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'blocks.0.0.conv_dw.weight', 'blocks.0.0.bn1.weight', 'blocks.0.0.bn1.bias', 'blocks.0.0.bn1.running_mean', 'blocks.0.0.bn1.running_var', 'blocks.0.0.se.conv_reduce.weight', 'blocks.0.0.se.conv_reduce.bias', 'blocks.0.0.se.conv_expand.weight', 'blocks.0.0.se.conv_expand.bias', 'blocks.0.0.conv_pw.weight', 'blocks.0.0.bn2.weight', 'blocks.0.0.bn2.bias', 'blocks.0.0.bn2.running_mean', 'blocks.0.0.bn2.running_var', 'blocks.0.1.conv_dw.weight', 'blocks.0.1.bn1.weight', 'blocks.0.1.bn1.bias', 'blocks.0.1.bn1.running_mean', 'blocks.0.1.bn1.running_var', 'blocks.0.1.se.conv_reduce.weight', 'blocks.0.1.se.conv_reduce.bias', 'blocks.0.1.se.conv_expand.weight', 'blocks.0.1.se.conv_expand.bias', 'blocks.0.1.conv_pw.weight', 'blocks.0.1.bn2.weight', 'blocks.0.1.bn2.bias', 'blocks.0.1.bn2.running_mean', 'blocks.0.1.bn2.running_var', 'blocks.0.2.conv_dw.weight', 'blocks.0.2.bn1.weight

In [16]:
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 2) 
